# Minor Flood Frequency

Writing words, still lots to do.

In [1]:
%run setup.ipynb
data_dir = Path('../data' )
output_dir = Path('../output') 

import calendar

In [2]:
# load the data
SL_daily_max = xr.open_dataset(data_dir / 'SL_daily_max.nc')
SL_daily_max.close()

hourly_data = xr.open_dataset(data_dir / 'SL_hourly_data.nc')
hourly_data.close()

In [3]:
SL_daily_max

rid = 57
station_name = SL_daily_max['station_name'].sel(record_id=rid).item()

# Make a dataframe with years as rows and months as columns
df = SL_daily_max['flood_day'].sel(record_id=rid).to_dataframe()
df = df.reset_index()
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df = df.drop(columns='time')

# sum the number of flood days in each month for each year
df = df.groupby(['year', 'month']).sum()

# pivot the table
df = df.pivot_table(index='year', columns='month', values='flood_day')

# rename months to month names
df.columns = calendar.month_abbr[1:]

# add a column for the total number of flood days in each year
df['Annual'] = df.sum(axis=1)



# add a row for the total number of flood days in each month, normalized the total flood days (last row, last column)
df.loc['Monthly Total (%)'] = df.sum()
df.loc['Monthly Total (%)'] = 100*df.loc['Monthly Total (%)']/df.loc['Monthly Total (%)', 'Annual']



tableTitle = station_name + ': Days Exceeding XX cm above MHHW'
# Apply background gradient and add a title with larger text
styled_df = df.style.background_gradient(
    cmap='Blues', axis=None, subset=(df.index[:-1], df.columns[:-1])
).format("{:.0f}").background_gradient(
    cmap='Purples', axis=None, subset=(df.index[-1], df.columns[:-1])
).format("{:.0f}").set_caption(tableTitle).set_table_styles([
    {
        'selector': 'caption',
        'props': [('font-size', '16px'), ('margin-bottom', '10px')]
    }
])

styled_df


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
year,,,,,,,,,,,,,
1983,6,0,0,0,0,1,0,6,0,0,0,0,13
1984,0,0,0,0,0,7,7,8,7,12,9,11,61
1985,12,3,0,0,0,3,5,1,0,0,1,0,25
1986,0,0,0,0,0,0,0,0,0,0,1,4,5
1987,2,0,0,0,0,0,0,0,0,0,4,11,17
1988,9,3,0,1,8,10,16,4,0,1,2,7,61
1989,0,2,0,0,0,2,3,0,0,3,4,5,19
1990,0,0,0,0,0,0,0,0,0,1,2,0,3
1991,3,0,0,0,0,0,1,6,2,0,6,5,23


In [4]:
#make a pretty pdf of the table with great_tables
from great_tables import GT,html

dfGT = df.copy()
dfGT['year'] = df.index

# put the year column first
cols = dfGT.columns.tolist()
cols = cols[-1:] + cols[:-1]
dfGT = dfGT[cols]
dfGT.reset_index(drop=True, inplace=True)



# Create a GreatTable object
table = (GT(dfGT)
.fmt_number(columns=calendar.month_abbr[1:13], decimals=0)
.fmt_number(columns=['Annual'], decimals=0)
.tab_header(title = 'Days Exceeding 30 cm above MHHW', subtitle = station_name)
.data_color(domain = [0,30],
            columns=calendar.month_abbr[1:13],
            rows = list(range(len(dfGT)-1)),
            palette=["white", "lightblue"])
.data_color(domain = [0,30],
            columns=calendar.month_abbr[1:13],
            rows = [-1],
            palette=["white", "purple"])
)

# save table as png
tableName = station_name + '_flood_days_intra_annual.png'
savePath = os.path.join(output_dir, tableName)

# replace any commas or spaces with underscores
savePath = savePath.replace(' ', '_')
table.save(savePath)

# Load Image
from IPython.display import Image
imgTable = Image(filename=savePath)

# Glue the image with a name
glue("imgTable", imgTable, display=False)



```{glue:figure} imgTable
:name: "table"
:align: center
:scale: 50%
Table of sea level records exceeding a given threshold. The data are organized into years, and months. Each 'event' is considered a 'flooding day,' where at least one hour of that day exceeds the threshold. The monthly percentage shown at the bottom of the table is relative to the total number of flooding days (for all months and years in the record). 
```